# Initiate
variable(value, list, and matrix) and function uses in neural network training and testing

In [ ]:
#import
import numpy as np
import random
import math

# Variable
global eta;
eta = 0.9;
# External Input based list
global xList;
xList = []
global dList;
dList = []
# Internal lists and matrix
global eList
eList = []
global biasList
biasList = [];
global wMatrix
wMatrix = [];
global vMatrix
vMatrix = []
global yMatrix
yMatrix = [];
global sigmaMatrix
sigmaMatrix = [];
global nodeIndexMatrix
nodeIndexMatrix = [];
# Class order. use in determine class encoded form
classOrder = ["cp","im","pp","imU","om","omL","imL","imS"];

## Functions
Following are the functions use during forward and backware propagation of the neural network

### Auxilary Function

In [ ]:
def getNodeLayer(nodeNum:int):
    # Return layer number of given node number
    # Parameter: interger representing node number
    # Raise Runtime error if given node does existed
    result = 0;
    for layer in nodeIndexMatrix:
        if nodeNum in layer: return result
        result += 1
    raise RuntimeError("no node " + nodeNum + "existed in nodeIdexMatrix");

def getNodeLayerNIndex(nodeNum:int):
    #Return layer number and index in said layer of given node number
    # Parameter: interger representing node number
    # Raise Runtime error if given node does existed
    nodeLayer = getNodeLayer(nodeNum)
    nodeIndex = nodeIndexMatrix[nodeLayer].index(nodeNum)
    return nodeLayer, nodeIndex

### Calculation Function
All following functions implemented using coordination-like node system and matrix

In [ ]:
def vFunc(nodeNum:int):
    # Return v value of given nodeNum.
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0:
        return 0;
    else:
        pastIndexes = nodeIndexMatrix[nodeLayer-1];
        pastWs = list(map(lambda x: wMatrix[x][nodeIndex], pastIndexes)) # get all left layer's weight associate with given nodeNum
        pastYs = yMatrix[nodeLayer-1] # get all left layer's y 
        wNyList = list(map(lambda x,y: x*y, pastYs,pastWs)) # calculate a sum of product of weight and its y.
        bias = biasList[nodeNum]; #get given node bias
        v = sum(wNyList)+bias; #calculate v
        return v;

def simgFunc(input:int):
    # Return result of given input's sigmoid activation function
    # Parameter: integer
    return 1/(1 + np.exp(-input));

def simgPrimeFunc(input:int):
    # Return result of given input's derivatived sigmoid activation calculation 
    # Parameter: integer
    return simgFunc(input)*(1-simgFunc(input));

def yFunc(nodeNum:int):
    # Return y value of given nodeNum (based on its v value).
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    # print("y index of:" + str(nodeNum))
    if nodeLayer == 0: # check if given node is input node
        return xList[nodeNum];
    else:
        v = vMatrix[nodeLayer][nodeIndex]; # get its calculated v values
        return simgFunc(v); # calculate and return sigmoid-ed v

def eFunc(nodeNum:int):
    # Return error of given nodeNum
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1: # check if given node is output node
        y = yMatrix[nodeLayer][nodeIndex] #get its y value
        d = dList[nodeIndex] # get its desire value
        return d - y;
    else:
        raise RuntimeError("Can't calculate e("+str(nodeNum)+")")

def sigmaFunc(nodeNum:int):
    # Return sigma or (E) of given nodeNum
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    phiPrime = simgPrimeFunc(vFunc(nodeNum))
    if nodeLayer == len(nodeIndexMatrix)-1: # check if given node is output node
        e = eFunc(nodeNum);
        return e*phiPrime;
    else:
        forWs = wMatrix[nodeNum]; # Get all its weigh with edge pointing to the right
        forSigmas = sigmaMatrix[nodeLayer+1] # get all sigma (E) values of right layer 
        sumSigW = sum(map(lambda x,y: x*y,forWs, forSigmas)); # calculate product of all weight and its pointed sigma
        return phiPrime*sumSigW; # calculate and return sigma 

def deltaWFunc(nodeI:int, nodeJ:int):
    # Return delta W of node I and J
    # Parameter: interger representing node I and J number
    nodeILayer, nodeIIndex = getNodeLayerNIndex(nodeI);
    nodeJLayer, nodeJIndex = getNodeLayerNIndex(nodeJ);
    yI = yMatrix[nodeILayer][nodeIIndex]; # get node I's y values
    if nodeJLayer == len(nodeIndexMatrix)-1: # Check if given node is output node
        e = eFunc(nodeJ);
        phiPrime = simgPrimeFunc(vFunc(nodeJ))
        return eta*e*phiPrime*yI;
    else:
        sigmaJ = sigmaMatrix[nodeJLayer][nodeJIndex];
        return eta*sigmaJ*yI;

def getNewWs(nodeNum:int):
    # Return new weight based on delta w of given nodeNUm
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1: # Check if given node is output node
        raise RuntimeError("Cann't calculate delta-Ws of output node:" + str(nodeNum))
    forNodeIndexes = nodeIndexMatrix[nodeLayer+1]; # get all node number of right layer
    oldWs = wMatrix[nodeNum] # get its weights associate with right layer
    deltaWs = list(map(lambda x: deltaWFunc(nodeNum, x), forNodeIndexes)) # get all delta w of said weights (oldWs)
    newWs = list(map(lambda x,y: x+y, oldWs,deltaWs)) # calculate all new weights
    return newWs;

def getNewBias(nodeNum:int):
    # Return new bias of given nodeNUm
    # Parameter: interger representing node number
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0: raise RuntimeError("No bias for node:"+str(nodeNum))  # if given node is input node raise runtime err
    sigma = sigmaMatrix[nodeLayer][nodeIndex]; # get sigma values of given node
    oldBias = biasList[nodeNum]; # get current bias of given node
    newBias = oldBias+(eta*sigma); # calculate new bias based on old bias
    return newBias;



# Training Section
Following functions design to simplify the neural network experiment

## Training Function
Functions uses as parts of trains and testing

In [ ]:
import itertools

def initAllContainer(layerNodeCount:list):
    # Clear and create all container use in the neural network
    # Parameter: a list of integer representing node count in each layer
     
    #Clear section
    xList.clear();
    dList.clear();
    eList.clear();
    biasList.clear();
    wMatrix.clear();
    vMatrix.clear()
    yMatrix.clear();
    sigmaMatrix.clear();
    nodeIndexMatrix.clear();
    #Container creation 
    currIndex = 0;
    for l in layerNodeCount:
        nodeIndexMatrix.append(list(range(currIndex,currIndex+l)))
        yMatrix.append([0]*l)
        vMatrix.append([0]*l)
        sigmaMatrix.append([0]*l)
        currIndex += l
    for layer in range(len(nodeIndexMatrix)):
        for node in nodeIndexMatrix[layer]:
            if not node in nodeIndexMatrix[len(nodeIndexMatrix)-1]:
                wMatrix.append([0]*len(nodeIndexMatrix[layer+1]))
    for _ in range(max(max(nodeIndexMatrix))+1):
        # eList.append(0);
        biasList.append(0);

def getRandomValue(minVal:float, maxVal:float):
    # Return a random any values but zero in given range
    # Parameters: minimal and maximal float values 
    result = 0
    while result == 0:
        result = round(random.uniform(minVal, maxVal),2)
    return result;

def UpdateRandomWMatrix(minVal:float, maxVal:float):
    # Update existed W Matrix with random value in given range
    # Parameters: minimal and maximal float values 
    for i in range(len(wMatrix)):
        for j in range(len(wMatrix[i])):
            wMatrix[i][j] = getRandomValue(minVal, maxVal)            

def UndateRandomBias(minVal:float, maxVal:float):
    # Update existed Bias list with random value in given range
    # Parameters: minimal and maximal float values 
    for i in range(len(biasList)):
        biasList[i] = getRandomValue(minVal, maxVal)  

def getXnDList(textsList:list):
    # Return a list of input and encode class
    # Parameter: a String representing tuple of data
    dataText = textsList;
    dataList = dataText.split();
    clas = dataList.pop(7)
    xL = list(map(float,dataList))
    dL = [0]*8;
    dL[classOrder.index(clas)] = 1;
    return xL, dL;

def forwardPropergate():
    # Update all v Matrix and y Matrix values in forward propagation manner
    for layNum in range(len(nodeIndexMatrix)):
        nodelist = nodeIndexMatrix[layNum]
        vList = list(map(vFunc, nodelist));
        vMatrix[layNum] = vList
        yList = list(map(yFunc, nodelist));
        yMatrix[layNum] = yList

def backPropergate():
    # Update all sigma Matrix, w Matrix, and bias list in back propagation manner
    reverseLayNum = list(range(len(nodeIndexMatrix))).copy()
    reverseLayNum.pop(0)
    reverseLayNum.reverse() # Create a list contain all node number in reverse manner
    # Update all sigma
    for layNum in reverseLayNum:
        nodelist = nodeIndexMatrix[layNum]
        sigmaList = list(map(sigmaFunc, nodelist));
        sigmaMatrix[layNum] = sigmaList
    # Update all W
    indexesForW = nodeIndexMatrix.copy()
    indexesForW.pop(len(nodeIndexMatrix)-1)
    for node in range(max(max(indexesForW))): # looping from least to most valued node number
        newWs = getNewWs(node);
        wMatrix[node] = newWs;
    # Update all Biase
    startBiasIndex = max(nodeIndexMatrix[0])
    for b in range(startBiasIndex+1,max(max(nodeIndexMatrix))+1): # Looping through hidden and output node
        newBias = getNewBias(b);
        biasList[b] = newBias;

def getResult(yList:list):
    # Return a predicted result based on given y / confident list
    # Parameters: list representing y list of output nodes
    result = [0]*8
    maxIndex = yList.index(max(yList));
    result[maxIndex] = 1
    return result

def grouper(inputList:list, n:int):
    # Return a list of size n group of given inputlist
    # Parameters:
    #  InputList: list to be regrouping
    #  n: integer representing size of most group
    args = [iter(inputList)] * n
    return list([e for e in t if e != None] for t in itertools.zip_longest(*args))

def getTrainNTestIndexesHoldUp(rowCount:int):
    # Return train and test data's indexes determining based on Hold-up method (randomly pick 90% of all indexes to be uses at train)
    allIndexes = range(rowCount);
    trainDataIndexes = random.sample(allIndexes, round(rowCount*0.9))
    testDataIndexes = [x for x in allIndexes if x not in trainDataIndexes]
    return trainDataIndexes, testDataIndexes

def getTrainNTestIndexes(rowCount:int, kNumber:int, kRound:int):
    # Return trian and test data's indexes of kRound fold.
    # Parameters:
    #  rowCount: integer representing total number of data instance
    #  kNumber: integer representing k in k-fold cross validation method
    #  kRound: integer representing k-fold of current validation
    indexesMatr = grouper(range(rowCount), round(rowCount/kNumber));
    testDataIndexes = indexesMatr[kRound]
    trainDataIndexes = [x for subList in indexesMatr for x in subList]
    trainDataIndexes = [x for x in trainDataIndexes if x not in testDataIndexes]
    return trainDataIndexes, testDataIndexes

## Execution

In [ ]:
def getNNResult(dataPath:str, NNDimen:list, learningRate:float, epoch:int, kNumber:int):
    # Return a list of accuracies percentage and losses of each fold test. Said tests are a part of neural network create based on given parameter.
    # Parameters:
    #   dataPath: String representing path to preprocessed data
    #   NNDimen: list of integer representing node count in each layer
    #  learningRate: float representing eta to be uses.
    #  epoch: integer representing how many each train dataset to be pass through neural network

    # init variable assoviate with dataset
    fil = open(dataPath, "r")
    texts = fil.readlines()
    random.shuffle(texts)
    rowCount = len(texts);

    # init variable to be use in neural network
    global eta;
    eta = learningRate;
    initAllContainer(NNDimen);
    UpdateRandomWMatrix(-0.1,0.1);
    UndateRandomBias(-0.1,0.1);
    lossList = []
    accList = []

    # Begin k - fold of train and test
    for k in range(kNumber):
        trainIndexes, testIndexes = getTrainNTestIndexes(rowCount, kNumber, k);
        # Train section
        for _ in range(epoch):
            global xList
            global dList
            for i in trainIndexes:
                xList, dList = getXnDList(texts[i]); # determining input and encode class
                forwardPropergate();
                backPropergate();
        
        # Test section
        corrCount = 0;
        totaLoss = 0
        for i in testIndexes:
            xList, dList = getXnDList(texts[i]); # determining input and encode class
            forwardPropergate();
            rawY = yMatrix[len(nodeIndexMatrix)-1] # get output layer y (confidence) values
            loss = sum(map(lambda x,y: x*math.log(y),dList, rawY)); # calculate loss based on predicted and actually result
            result = getResult(rawY);  # get the predicted result
            if (dList==result): corrCount += 1; # correct counting
            totaLoss += loss;
        lossList.append((-1/len(testIndexes))*totaLoss); # calculate and record loss of current fold
        accList.append(corrCount/len(testIndexes)*100); # calculate and record accuracy of current fold
    return accList, lossList;

# Experimenting Section
A section for config all neural network parameter such as datapath, architecture, learning rate run begin the experiment. Follow by a plotting of line graph representing accuracy and loss of recorded experiment 

In [ ]:
import matplotlib.pyplot as plt

resultAccs, resultLosses = getNNResult("ProcessedData/type3data.data", [7,8,8],0.01,50,10);
# Accuracy Graph creation
plt.title("Accuracy")
plt.ylabel("Accuracy(%)")
plt.xlabel("fold")
temp = list(map(lambda x: round(x,3), resultAccs))
plt.plot(temp)
plt.show()
# Loss graph creation
plt.title("Categorical Cross-Entropy loss")
plt.ylabel("loss")
plt.xlabel("fold")
temp = list(map(lambda x: round(x,3), resultLosses))
plt.plot(temp)
plt.show()
